In [4]:
API_KEY = "2RT8mQ01jQmMrZwiHczy75cckYTP2Kii9McSvgDCGFJ8Q7P37vVwJQQJ99BJACYeBjFXJ3w3AAAAACOGxEp9"

In [ ]:
import os
from openai import AzureOpenAI

endpoint = "https://chr-umn-hackathon-resource.cognitiveservices.azure.com/"
model_name = "gpt-5-mini"
deployment = "gpt-5-mini"

subscription_key = "2RT8mQ01jQmMrZwiHczy75cckYTP2Kii9McSvgDCGFJ8Q7P37vVwJQQJ99BJACYeBjFXJ3w3AAAAACOGxEp9"
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "I am going to Paris, what should I see?",
        }
    ],
    # max_completion_tokens=16384,
    model=deployment
)

print(response.choices[0].message.content)


Nice — Paris has a lot to offer. A few quick questions so I can tailor recommendations: how many days will you be there, what are your main interests (art, food, history, architecture, shopping, kids), and do you want classic sights or hidden/locals’ spots?

Meanwhile, here are high‑value suggestions, practical tips, and sample itineraries.

Top must-sees
- Eiffel Tower — best seen at dusk (book tickets for summit or second floor in advance). Trocadéro offers great photo views.
- Louvre — world’s largest art museum (Mona Lisa, Venus de Milo). Buy timed-entry tickets ahead, it’s closed Tuesdays.
- Musée d’Orsay — Impressionist masterpieces in a former train station (closed Mondays).
- Notre‑Dame & Île de la Cité — cathedral exterior (restoration ongoing), plus nearby Sainte‑Chapelle (stained glass).
- Montmartre & Sacré‑Cœur — charming hilltop neighborhood, artists’ square, panoramic city views.
- Champs‑Élysées & Arc de Triomphe — shopping and climb the Arc for a central view.
- Le Mar

In [6]:
import os
from openai import AzureOpenAI

from typing import Annotated

from langchain.chat_models import init_chat_model
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

os.environ["AZURE_OPENAI_API_KEY"] = API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://chr-umn-hackathon-resource.cognitiveservices.azure.com/"
os.environ["OPENAI_API_VERSION"] = "2025-03-01-preview"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-5-mini"

llm = init_chat_model(
    "azure_openai:gpt-5-mini",
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
)


def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile()

In [7]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    print("No")
    pass

No


In [16]:



def stream_graph_updates(user_input: str):
    print("User:", user_input)
    for event in graph.stream({"messages": [{"role": "user", "content": user_input},
                                            {"role": "system",
                                             "content": "You are a helpful assistant that handle negotiating the rates to haul a load"},
                                             {"role": "system",
                                              "content": '''The user shou tell you the load which include: depature place and detination;
                                              Commodity and its special needs(like reefer required), weight,distance and target prize. If they
                                              don't provide this, you shoud ask them to provide these information.
                                                '''},
                                            {"role":"system", "content":''' Please talk like a human, don't ask too much at a time'''}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

User: I want to loas from Atlanta, GA → Chicago, IL
Assistant: Got it — Atlanta, GA → Chicago, IL. That route is roughly 710–730 driving miles (about 10–12 hours).

I can start negotiating rates, but I need a few quick details first. Can you tell me:

1) What’s the commodity and any special needs (reefer, hazmat, liftgate, palletized, fragile, etc.)?  
2) Total weight, number of pieces/pallets, and dimensions if you have them.  
3) Pickup date/time and required delivery date/window. Do you have a preferred equipment type (dry van, reefer, flatbed, step deck)?  
4) What’s your target price (total or $/mile), or the best rate you’re willing to accept?

Give me those and I’ll work the rates. If you don’t know something, just say so and I’ll advise what’s typical.
Goodbye!
